# Обработка исходных файлов. Удаление и парсинг

In [1]:
import glob
from xml.dom import minidom
import os


def get_fb2_filenames():
    return glob.iglob('/mnt/data/**/*.fb2', recursive=True)


def get_title_info(node):
    description = node.getElementsByTagName('title-info')
    assert len(description) == 1, 'только одно описание'
    
    return description[0]


def get_genres(node):
    genre = node.getElementsByTagName('genre')

    return {g.childNodes[0].data for g in genre if len(g.childNodes)}


def find_lang(node):
    lang = node.getElementsByTagName('lang')
    assert len(lang) <= 1, 'указано небольше одного языка'
    
    return lang[0].childNodes[0].data if len(lang) and len(lang[0].childNodes) else None


def get_title(node):
    title = node.getElementsByTagName('book-title')
    assert len(title) == 1 and len(title[0].childNodes) == 1, 'указано название книги'
    
    return title[0].childNodes[0].data


def is_children_genre(genres):
    # Обучение про секс и проч.
    if 'child_education' in genres:
        return False
    
    for g in genres:
        if 'child' in g:
            return True
 
    return False


def get_filenames(fb2_filename):
    base = os.path.splitext(fb2_filename)[0]
    
    return {
        "fb2": fb2_filename,
        "txt": f"{base}.txt",
        "stanza": f"{base}.stanza",
    }


def get_text(doc):
    result = []
    
    for body in doc.getElementsByTagName('body'):
        for paragraph in body.getElementsByTagName('v'):
            collect_text(paragraph, result)
        
        for paragraph in body.getElementsByTagName('p'):
            if paragraph.parentNode.tagName not in ['section', 'cite']:
                continue
                
            collect_text(paragraph, result)

    return ' '.join(result)


def get_author(node):
    f = node.getElementsByTagName('first-name')
    m = node.getElementsByTagName('middle-name')
    l = node.getElementsByTagName('last-name')
    
    text = []
    
    if len(f) and f[0]:
        collect_text(f[0], text)
        
    if len(m) and m[0]:
        collect_text(m[0], text)
        
    if len(l) and l[0]:
        collect_text(l[0], text)
    
    return ' '.join(text).replace('  ', ' ') if len(text) else None


def collect_text(node, result):
    if node.nodeType == node.TEXT_NODE:
        result.append(node.data)
        
    for child in node.childNodes:
        collect_text(child, result)
        

def read_txt(fb2_filename):
    txt_filename = get_filenames(fb2_filename)["txt"]
    
    try:
        with open(txt_filename, 'r') as file:
            return file.read()
    except:
        return None


def write_txt(fb2_filename, text):
    txt_filename = get_filenames(fb2_filename)["txt"]
    
    with open(txt_filename, 'w') as file:
        return file.write(text)

        
def delete_book(filename):
    fns = get_filenames(filename)
    
    for key in fns:
        if os.path.isfile(fns[key]):
            os.remove(fns[key])

# Удаляем книги по описанию

In [40]:
BAD_AUTHORS = {
    'Александр Мазин',
    'Алексей Слаповский',
    'Анастасия Метельская',
    'Анатолий Голубев',
    'Андреа Робинсон',
    'Андрей Ветер',
    'Андрей Михайлович Зинчук',
    'Анна Гурова',
    'Анника Тор',
    'Бет Хилгартнер',
    'Борис Егорович Бондаренко',
    'Борис Николаевич Сергуненков',
    'Валерий Роньшин',
    'Вера Сергеевна Аксенова',
    'Вера и Марина Воробей',
    'Виктор Крамаренко',
    'Виктор Федорович Авдеев',
    'Владимир Александрович Вафин',
    'Галина Гордиенко ',
    'Денис Белохвостов',
    'Джеки Л. Даррел',
    'Джеральд Даррел',
    'Джин Уэбстер',
    'Джон Томпсон',
    'Дмитрий Глуховский',
    'Дмитрий Миронович Молдавский',
    'Дмитрий Санин',
    'Екатерина Александровна Неволина',
    'Екатерина Вадимовна  Вадимовна  Мурашова',
    'Екатерина Вильмонт',
    'Елена Александровна Матвеева',
    'Елена Александровна Усачева',
    'Жаклин Уилсон',
    'Ирина Медведева',
    'Ирина Сабурова',
    'Кир Булычев',
    'Лилиан Джексон Браун',
    'Любовь Валерьевна Романова',
    'Людвик Сергеевич Ашкенази',
    'Макс Фрай',
    'Максуд Ибрагимбеков',
    'Марина Семеновна Аромштам',
    'Мариэтта Омаровна Чудакова',
    'Мария Митрофанова',
    'Мелвин Берджес',
    'Мелиса И. Холмс',
    'Михаил Пришвин',
    'Н. В. Ковалева',
    'Н. Семенюта',
    'Надежда Опрышко',
    'Наталья Брониславовна Шешко',
    'Николай Игнатьевич Киселев-Громов',
    'Никос Зервас',
    'Олег Николаевич Верещагин',
    'Олег Раин',
    'Павел Александрович Алексеев',
    'Патрик Несс',
    'Равиль Нагимович Бикбаев ',
    'Радий Петрович Погодин',
    'Роман Волков',
    'Роман Львович Шмараков',
    'Сергей Гусаков',
    'Сергей Сухинов',
    'Сильвер Равенвольф',
    'Чарльз де Линт',
    'Чэнь Мяо',
    'Эдуард Мартинович Скобелев',
    'Эрик Ниланд',
    'Илья Туричин',
    'Михаил Антонович Алпатов',
    'Кирилл Кащеев',
    'Дмитрий Васильевич Бахта',
    'Яник Городецкий',
    'Диана Уинн Джонс',
    'Владимир Марченко',
    'Марк Уолден',
    'Александрович без автора',
    'Александр Иванович Папченко',
    'Екатерина Николаевна Вильмонт',
    'Игорь Дмитриевич Востряков',
    'Кирилл Кащеев',
    'Виктор Селезнёв',
    'Валерий Тимофеев',
    'Алексей Биргер',
    'Василий Тихонов',
    'Альберт Анатольевич Лиханов',
    'Автор неизвестен',
    'Сильвия Паола Каваллано',
    'Борис Степанович Житков',
    'Игорь Минутко',
    'Рустам Карапетьян',
    'Людмила Петрушевская',
    'Дмитрий Щеглов',
    'Борис Карлов',
    'Мони Нильсон-Брэнстрем',
    'Ахто Леви',
    'Эндре Люнд Эриксен',
    'Стивен Кинг',
    'Наталья Игоревна Романова',
    'Сергей Васильевич Максимов',
    'Наталья Иванова',
    'Жаклин  Уилсон',
    'Ольга Клюкина',
    'Андреас Штайнхёфель',
    'Игорь Евгеньевич Всеволожский',
    'Федор Васильевич Гладков',
    'Фазиль Абдулович Искандер',
    'Джеральд С. Даррел',
    'Юн Эво',
    'Сфинкс',
    'Рик Риордан',
    'Айрин Эльба',
    'Ханс-Георг Ноак',
    'Наталья Игоревна Романова',
    'Герман Иванович Матвеев',
    'Лидия Алексеевна Чарская',
    'Карен М. Маккомби',
    'Илья Владимирович Бояшов',
    'Павел Карпов',
    'Энн Файн',
    'Арсений Иванович Рутько',
    'Сергей Анатольевич Иванов',
    'Anya Shinigami',
    'Беате Тереза Ханика',
    'Харри Владимирович Йыгисалу',
    'Франсуа Пети де Ла Круа',
    'Ирина Щеглова',
    'Нэнси Борисовна Фармер',
    'Владимир Кузьмин',
    'Сергей Веселов',
    'Михаил Лезинский',
    'Теодор Львович Вайсенборн',
    'Шамиль Зиганшинович Ракипов',
    'Эдуард Басс',
    'Алексей Иванович Слаповский',
    'Юрий Германович Вебер',
    'без автора',
    'Юрий Иосифович Коринец',
    'Юрий Яковлев',
    'Клара Ярункова',
    'Михаил Евграфович Салтыков-Щедрин',
    'Мария Грипе',
    'Галина Гордиенко',
    'Роберт Хайнлайн',
    'Серафима Константиновна Власова',
    'Николай Петрович Вагнер',
    'Кристине Нёстлингер',
    'Наталья Викторовна Шерешевская',
    'Джеральд Л. Даррел',
    'KateRon',
    'Наталья Васильевна Щерба',
    'Ирина Мазаева',
    'Георгий Филиппович Шолохов-Синявский',
    'Андрис Пуриньш',
    'Андрей Саломатов',
}


def delete_books_by_info():
    count = 0
    
    for filename in get_fb2_filenames(): 
        try:
            count += 1
            if count % 10 == 0: print('№', count)
                
            book = minidom.parse(filename)
            info = get_title_info(book)
            genres = get_genres(info)

            if not is_children_genre(genres):
                print('del genre', get_title(info), genres)
                delete_book(filename)
                continue

            lang = find_lang(info)

            if not lang or lang.lower() != 'ru':
                print('del lang', get_title(info), lang)
                delete_book(filename)
                continue
                
            author = get_author(info)
            
            if author and author in BAD_AUTHORS:
                print('del author', get_title(info), author)
                delete_book(filename)
                continue
                
            title = get_title(info)
            
            if 'на украинском языке' in title:
                print('del title', title)
#                 delete_book(filename)
                continue

        except Exception as e:
            print(filename, e)
            
delete_books_by_info()

№ 10
№ 20
№ 30
№ 40
№ 50
№ 60
№ 70
№ 80
№ 90
№ 100
№ 110
№ 120
№ 130
№ 140
№ 150
№ 160
№ 170
№ 180
№ 190
№ 200
№ 210
№ 220
№ 230
№ 240
№ 250
№ 260
№ 270
№ 280
№ 290
№ 300
№ 310
№ 320
№ 330
№ 340
№ 350
№ 360
№ 370
№ 380
№ 390
№ 400
№ 410
№ 420
№ 430
№ 440
№ 450
№ 460
№ 470
№ 480
№ 490
№ 500
del author Танатос Андрей Саломатов
№ 510
№ 520
№ 530
del title Рiка далеких мандрiв (на украинском языке)
№ 540
№ 550
№ 560
№ 570
№ 580
del title Забутi письмена (на украинском языке)
№ 590
№ 600
№ 610
№ 620
№ 630
№ 640
№ 650
№ 660
№ 670
№ 680
№ 690
№ 700
№ 710
№ 720
№ 730
№ 740
№ 750
№ 760
№ 770
№ 780
№ 790
№ 800
№ 810
№ 820
№ 830
№ 840
№ 850
№ 860
№ 870
№ 880
№ 890
№ 900
№ 910
№ 920
№ 930
№ 940
№ 950
№ 960
№ 970
№ 980
№ 990
№ 1000
№ 1010
№ 1020
№ 1030
№ 1040
№ 1050
№ 1060
№ 1070
№ 1080
del title Дивовижнi пригоди капiтана мiжзоряного плавання Небрехи (на украинском языке)
№ 1090
№ 1100
del title Народження Адама (на украинском языке)
del title Бiблiйнi пригоди на небi i на землi (на украинском язы

# Удаляем книги по содержимому

In [39]:
from collections import Counter


def delete_books_by_text():
    count = 0
    counter = set()
    
    for filename in get_fb2_filenames(): 
        try:
            count += 1
            if count % 100 == 0:
                print('№', count)
                
            text = read_txt(filename)
            book = None
            
            if not text:
                book = minidom.parse(filename)
                text = get_text(book)
                write_txt(filename, text)
                
            
            
            
            
#             book = book or minidom.parse(filename)
#             info = get_title_info(book)
#             author = get_author(info)
            
#             counter[author] += 1
            
#             continue
            
            
            

                
            bad_words = [
                ' секс ',
                'гомосек',
                'пидор',
                'педера',
                'проститу',
                ' cука ',
                ' хуй',
                'педофил',
                'педофил',
                ' хуе',
                'пизд',
                'трахат',
                'трахаю',
                'трахая',
                'блядь',
                ' ебать',
                'пизд',
                'изнасил',
                'изнасилов',
                'влагал',
                ' пенис ',
                'вагин',
                ' анальн',
                ' сиськ',
                'онани',
                'мустурб',
                'оргазм',
                'шлюх',
                'некрофил',
                'зоофил',
            ]
            
            for bw in bad_words:
                index = text.find(bw)

                if index == -1:
                    continue

                [pre, post] = [
                    text[max(0, index - 64):index],
                    text[index + len(bw):min(len(text), index + 64)]
                ]
                
                book = book or minidom.parse(filename)
                info = get_title_info(book)
                title = get_title(info)
                genres = get_genres(info)
                author = get_author(info)
                
                print('\n', title, genres, author)
                print(f"{pre}__{bw}__{post}")
                counter.add(author)

            

        except Exception as e:
            print(filename, e)
            
    return counter
            

authors_countres = delete_books_by_text()

authors_countres

№ 100
№ 200
№ 300
№ 400
№ 500

 Танатос {'child_sf'} Андрей Саломатов
нате, Алтухов услышал довольно приятный голос соседки. – У этой __шлюх__и опять новый мужик, – сообщила она кому-то. Алтухов захлопн

 Забутi письмена (на украинском языке) {'children'} Богдан Сушинский
ати що це був саме космодром.  - Так, нiхто. Але я глибоко перек__онани__й у цьому. Передбачаючи загибель Атлантиди, називатимемо її
№ 600
№ 700
№ 800

 Суперклей Христофора Тюлькіна, або “Вас викрито - здавайтесь!” {'children', 'sf_humor'} Анатолий Григорьевич Костецкий
д природи, а от силу волi можна в собi виховати. I вiн був перек__онани__й: рано чи пiзно Христофор неодмiнно винайде щось дуже й ду
№ 900
№ 1000
№ 1100

 Народження Адама (на украинском языке) {'children'} Юрий Ячейкин
ндрам, вашi бiлки, безумовно, прокисли б. Та, як бачите, я перек__онани__й прихильник старих залiзних традицiй.  - Ваша правда, - зм

 Спалах понадновот зiрки (на украинском языке) {'children'} Юрий Ячейкин
и.  А на ранок мене чекав сю

{None,
 'Анатолий Григорьевич Костецкий',
 'Андрей Саломатов',
 'Богдан Сушинский',
 'Владимир Михайлович Сотников',
 'Дмитрий Александрович Емец',
 'Игорь Александрович Минутко',
 'Ирмгард Койн',
 'Роберт Лоуренс Стайн',
 'Фред Адра',
 'Юрий Ячейкин',
 'Януш Борисовна Корчак'}

# Парсим синтаксис

In [ ]:
from nlp import nlp
import torch


def parse_syntax():
    count = 0
    
    for filename in get_fb2_filenames():
        stanza_filename = get_filenames(filename)["stanza"]
        
        try:
            count += 1
            if count % 10 == 0: print('№', count)

            if os.path.isfile(stanza_filename):
                continue

            text = read_txt(filename)
            
            if not text:
                text = get_text(minidom.parse(filename))
                write_txt(filename, text)

            with torch.no_grad():
                with open(stanza_filename, 'wb') as file:
                    nlp_doc = nlp(text)
                    file.write(nlp_doc.to_serialized())

        except Exception as e:
            print(filename, e)
            
parse_syntax()

2022-03-27 20:18:16 INFO: Downloading these customized packages for language: ru (Russian)...
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| pretrain  | syntagrus |

2022-03-27 20:18:16 INFO: File exists: /home/kukuruku/stanza_resources/ru/tokenize/syntagrus.pt.
2022-03-27 20:18:16 INFO: File exists: /home/kukuruku/stanza_resources/ru/pos/syntagrus.pt.
2022-03-27 20:18:16 INFO: File exists: /home/kukuruku/stanza_resources/ru/lemma/syntagrus.pt.
2022-03-27 20:18:18 INFO: File exists: /home/kukuruku/stanza_resources/ru/depparse/syntagrus.pt.
2022-03-27 20:18:19 INFO: File exists: /home/kukuruku/stanza_resources/ru/pretrain/syntagrus.pt.
2022-03-27 20:18:19 INFO: Finished downloading models and saved to /home/kukuruku/stanza_resources.
2022-03-27 20:18:19 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | synta

№ 10
№ 20
№ 30
№ 40
№ 50
№ 60
№ 70
№ 80
№ 90
№ 100
№ 110
№ 120
№ 130
№ 140
№ 150
№ 160
№ 170
№ 180
№ 190
№ 200
№ 210
№ 220
№ 230
№ 240
№ 250
№ 260
№ 270
№ 280
№ 290
№ 300
№ 310
№ 320
№ 330
№ 340
№ 350
№ 360
№ 370
№ 380
№ 390
№ 400
№ 410
№ 420
№ 430
№ 440
№ 450
№ 460
№ 470
№ 480
№ 490
№ 500
№ 510
№ 520
№ 530
№ 540
№ 550
№ 560
№ 570
№ 580
№ 590
№ 600
№ 610
№ 620
№ 630
№ 640
№ 650
№ 660
№ 670
№ 680
№ 690
№ 700
№ 710
№ 720
№ 730
№ 740
№ 750
№ 760
№ 770
№ 780
№ 790
№ 800
№ 810
№ 820
№ 830
№ 840
№ 850
№ 860
№ 870
№ 880
№ 890
№ 900
№ 910
№ 920
№ 930
№ 940
№ 950
№ 960
№ 970
/mnt/data/407046-407192/407047.fb2 no element found: line 10771, column 6
№ 980
№ 990
№ 1000
№ 1010
№ 1020
№ 1030
№ 1040
№ 1050
№ 1060
№ 1070
№ 1080
№ 1090
№ 1100
№ 1110
№ 1120
№ 1130
№ 1140
№ 1150
№ 1160
№ 1170
№ 1180
№ 1190
№ 1200
№ 1210
№ 1220
№ 1230
№ 1240
№ 1250
№ 1260
№ 1270
№ 1280
№ 1290
№ 1300
№ 1310
№ 1320
№ 1330
№ 1340
№ 1350
№ 1360
№ 1370
№ 1380
№ 1390
№ 1400
№ 1410
№ 1420
№ 1430
№ 1440
№ 1450
№ 1460
№ 1470
№ 148

False